<div style="background:#FFFFAA">
<img src="logo.jpg", width=150, ALIGN="left", border=20>
<center>
<h1>Sample Starting Kit </h1>
 <br>This code was tested with <br>
Python 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) (https://anaconda.org/)<br>
<i> Adapted for Chalab by Isabelle Guyon from original code of Balázs Kégl</i> <br>
<a href="http://www.datascience-paris-saclay.fr">Paris Saclay Center for Data Science (CDS)</a>
</center>
<br>
<br>
<br>
<p>
ALL INFORMATION, SOFTWARE, DOCUMENTATION, AND DATA ARE PROVIDED "AS-IS". The CDS, CHALEARN, AND/OR OTHER ORGANIZERS OR CODE AUTHORS DISCLAIM ANY EXPRESSED OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR ANY PARTICULAR PURPOSE, AND THE WARRANTY OF NON-INFRIGEMENT OF ANY THIRD PARTY'S INTELLECTUAL PROPERTY RIGHTS. IN NO EVENT SHALL AUTHORS AND ORGANIZERS BE LIABLE FOR ANY SPECIAL, 
INDIRECT OR CONSEQUENTIAL DAMAGES OR ANY DAMAGES WHATSOEVER ARISING OUT OF OR IN CONNECTION WITH THE USE OR PERFORMANCE OF SOFTWARE, DOCUMENTS, MATERIALS, PUBLICATIONS, OR INFORMATION MADE AVAILABLE FOR THE CHALLENGE. 
</div>

<div style="background:#FFFFAA">
    <h2>Introduction </h2>
    <p>
In an era where computer graphics techniques for image generation are reaching stunning levels of quality, it becomes more and more challenging to detect fake from true, authentic images. However, this raises a lot of legal issues, mainly dealing with forgery. This project focuses mainly on the efficiency of Generative Adversarial Network (GAN) algorithms for producing art forgery and we will try to beat the state-of-the-art models that detect it. In fact, the Fine Arts Expert Institute in Geneva estimates that as much as 50 percent of artworks currently in circulation may be forgeries. Although the standard approach for image classification which is deep neural networks and especially Convolutional Neural Networks (CNNs) works very well nowadays, GAN performs strongly on generating fake images. By focusing on the art forgery aspect, we will try to bring a more specific perspective on the issue of image forgery.

</div>

### Requirements
Our code uses multiple libraries, so the next cell will install python's required dependencies. In case you don't want to, or are running in the competition's docker, you can comment it.

In [20]:
#!pip install --user -r requirements.txt

In [21]:
%reload_ext autoreload
model_dir = 'sample_code_submission/'                        # Change the model to a better one once you have one!
result_dir = 'sample_result_submission/' 
problem_dir = 'ingestion_program/'  
score_dir = 'scoring_program/'
from sys import path; path.append(model_dir); path.append(problem_dir); path.append(score_dir); 
%matplotlib inline
%load_ext autoreload
%autoreload 2
import seaborn as sns; sns.set()
import warnings
import os
warnings.simplefilter(action='ignore', category=FutureWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<div style="background:#FFFFAA">
    <h1> Step 1: Exploratory data analysis </h1>
<p>
We provide sample_data with the starting kit, but to prepare your submission, you must fetch the public_data from the challenge website and point to it.
    <br>
    <span style="color:red"> Just change the data name in the block below. In the rest of the section, replace the sample plots by anything you want. </span>
</div>

In [22]:
data_dir = '../public_data'         
data_name = 'perso'
!ls $data_dir

perso_feat.name
perso_label.name
perso_public.info
perso_test.data
perso_train.data
perso_train.solution
perso_valid.data


In [23]:
from data_io import read_as_df
data = read_as_df(data_dir  + '/' + data_name)          # The perso_data is loaded as a Pandas Data Frame

Reading ../public_data/perso_train from AutoML format
Number of examples = 65856
Number of features = 200
   Class
0  False
1   True
Number of classes = 2


In [24]:
#data.head()

For convenience, we load the data as a "pandas" data frame, so we can use "pandas" and "seaborn" built in functions to explore the data.

In [25]:
#data.describe() 

<div style="background:#FFFFAA">
<h1>Step 2: Building a predictive model</h1>
</div>

<div style="background:#FFFFAA">
    <h2>Loading data with DataManager</h2>
    <p>
We reload the data with the AutoML DataManager class because this is more convenient:
  
</div>

In [26]:
from data_manager import DataManager
D = DataManager(data_name, data_dir, replace_missing=True)
#print(D)
print(data_name)

Info file found : C:\Users\isabe\Pictures\monet-master\monet-master\public_data\perso_public.info
perso


<div style="background:#FFFFAA">
    <h2>Training a predictive model</h2>
    <p>
We provide an example of predictive model (for classification or regression) in the `sample_code_submission/` directory. It is a quite stupid model: it makes constant predictions. Replace it with your own model.
    </div>

In [39]:
from numpy.core.umath_tests import inner1d
from data_io import write
from model import model

<div style="background:#FFFFAA">
an instance of the model (run the constructor) and attempt to reload a previously saved version from `sample_code_submission/`:
    
</div>

In [40]:
M = model()
trained_model_name = model_dir + data_name
print(data_name)
print(model_dir)
print(trained_model_name)

perso
sample_code_submission/
sample_code_submission/perso


<div style="background:#FFFFAA">
    Train the model (unless you reloaded a trained model) and make predictions. 
</div>

<div style="background:#FFFFAA">
    <b> Save the trained model </b> (will be ready to reload next time around) and save the prediction results. IMPORTANT: if you save the trained model, it will be bundled with your sample code submission. Therefore your model will NOT be retrained on the challenge platform. Remove the pickle from the submission if you want the model to be retrained on the platform.
</div>

In [41]:
"""
on a choisis de modifer que les paramétres ayant un impact sur la classication comme : 
_ n_estimators = nbre d'arbre plus il est élevé mieux et plus stable seront les predictions 
_ max_features = Voici le nombre maximal de features que Random Forest est autorisé à essayer dans une arborescence 
  individuelle.Ces fonctions améliorent la performence du model à chaque noeud mais cella ralentie aussi la tache
_ max_depth = la largeur d'un arbre pouvant être aussi le nombre de ces branches 

Pour cella nous avions essayer deux methodes principals qui sont la Random Search et la Grid Search
Ces dernieres nous donne les meilleurs parametres de facon plus ou moins rapide que on a ensuite remplacer dans 
le model clasifieur du model.py 
Nous avions commenter ces codes car ils prennent énornement de temps entre 45 min et 1h pour les hautes valeurs 
"""
"""
    # Recherche des meilleurs paramétre avec la methode Random Search 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV 
from time import time
# Create a based model
clf = MLPClassifier()

# Create the parameter grid based on the results of random search 
param_grid = { 
            'activation': ['identity', 'logistic', 'tanh', 'relu'],
            'max_iter': [100,500,1000,1500,2000],
            'solver': ['lbfgs', 'sgd', 'adam'],
            'learning_rate': ['invscaling', 'adaptive'],
            'hidden_layer_sizes': [(100,50,20,50),(100,50,20,50,20), (100,50,100,50,20),(100,50)],
            }
            
# Instantiate the grid search model
n_iter_search = 20

random_search = RandomizedSearchCV(clf, param_grid,  n_iter=n_iter_search, cv=5)


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


start = time()
random_search.fit(D.data['X_train'], D.data['Y_train'])
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)
"""
"""    # Recherche des meilleurs paramétre avec la methode Grid Search
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV 

# Create a based model
clf = MLPClassifier()

# Create the parameter grid based on the results of Grid Search 
param_grid = { 
            'activation': [‘identity’, ‘logistic’, ‘tanh’, ‘relu’],
            'max_iter': [100,500,1000,1500],
            'solver': [‘lbfgs’, ‘sgd’, ‘adam’],
            'learning_rate': [‘constant’, ‘invscaling’, ‘adaptive’],
            'hidden_layer_sizes': [(100,50,20,50),(100,50,20,50,20), (100,50,100,50,20), (200,100,50,20)],
            }
# Instantiate the grid search model
grid_search1 = GridSearchCV(clf, param_grid) """

if not(M.is_trained):
    X_train = D.data['X_train']
    Y_train = D.data['Y_train']
    # Fit the grid search to the data
    #grid_search.fit(X_train, Y_train)
    
    # affiche les meilleurs parametres 
    #print(grid_search.best_params_)
    M.fit(X_train, Y_train)                     

Y_hat_train = M.predict(D.data['X_train'])
Y_hat_valid = M.predict(D.data['X_valid'])
Y_hat_test = M.predict(D.data['X_test'])
if(os.path.exists("/home/tp-home005/nmbarak/projects/monet/public_data/perso_train.solution")) : 
    Y_test = D.data['Y_test']
    Y_valid = D.data['Y_valid']


FIT: dim(X)= [65856, 200]
FIT: dim(y)= [65856, 1]
PREDICT: dim(X)= [65856, 200]
PREDICT: dim(y)= [65856, 1]
PREDICT: dim(X)= [9408, 200]
PREDICT: dim(y)= [9408, 1]
PREDICT: dim(X)= [18817, 200]
PREDICT: dim(y)= [18817, 1]


In [42]:
print(Y_hat_train.shape)

(65856,)


In [43]:
M.save(trained_model_name)                 
result_name = result_dir + data_name
from data_io import write
write(result_name + '_train.predict', Y_hat_train)
write(result_name + '_valid.predict', Y_hat_valid)
write(result_name + '_test.predict', Y_hat_test)
!ls $result_name*

sample_result_submission/perso_test.predict
sample_result_submission/perso_train.predict
sample_result_submission/perso_valid.predict


In [44]:
print(len(Y_hat_train))



65856


<div style="background:#FFFFAA">
    <h2>Scoring the results</h2>
    <h3>Load the challenge metric</h3>
</div>

#### Metric : AUC_metric 
ROC curves are typically used in binary classification to study the output of a classifier

In [45]:
from libscores import get_metric
metric_name, scoring_function = get_metric()
print('Using scoring metric:', metric_name)

Using scoring metric: roc_auc_score


<div style="background:#FFFFAA">
    <h3> Training performance </h3>
    <p>
The participants normally posess target values (labels) only for training examples (except for the sample data). We compute with the `example` metric the training score, which should be zero for perfect predictions.
        </div>

In [46]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
n_classes=2
def fpr_tpr(solution, prediction):
    for i in range(n_classes):
        fpr, tpr, _ = metrics.roc_curve(solution, prediction)
        roc_auc = metrics.auc(fpr, tpr)
    return (fpr,tpr)
def p2c(prediction,threshold=0.5) : 
    c = []
    for ele in prediction : 
        if(ele>=0.5) : 
            c.append(1)
        else : 
            c.append(0)
    return np.array(c)

def plot_cm_matrix(solution,prediction,title) :
    prediction = p2c(prediction)
    cm = confusion_matrix(solution, prediction)
    df_cm = pd.DataFrame(cm, index = [i for i in "01"],columns = [i for i in "01"])
    plt.figure(figsize = (5,3))
    sn.heatmap(df_cm, annot=True)
    plt.title(title)

def plot_ROC(fpr,tpr,title) :
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',lw=lw)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.show()

if (os.path.exists("/home/tp-home005/nmbarak/projects/monet/public_data/perso_train.solution")): 
    fpr_train, tpr_train = fpr_tpr(Y_train, Y_hat_train)
    fpr_test,tpr_test = fpr_tpr(Y_test, Y_hat_test)
    fpr_valid,tpr_valid = fpr_tpr(Y_valid, Y_hat_valid)

    print('Training score for the', metric_name, 'metric = %5.4f' % scoring_function(Y_train, Y_hat_train))
    print('Ideal score for the', metric_name, 'metric = %5.4f' % scoring_function(Y_train, Y_train))
    print('Test score for the', metric_name, 'metric = %5.4f' % scoring_function(Y_test, Y_hat_test))
    print('Valid score for the', metric_name, 'metric = %5.4f' % scoring_function(Y_valid, Y_hat_valid))

    plot_cm_matrix(Y_train,Y_hat_train,"Confusion matrix for train data") 
    plot_ROC(fpr_train,tpr_train,"ROC curve for train data")
    plot_cm_matrix(Y_test,Y_hat_test,"Confusion matrix for test data") 
    plot_ROC(fpr_test,tpr_test,"ROC curve for test data")
    plot_cm_matrix(Y_valid,Y_hat_valid,"Confusion matrix for valid data") 
    plot_ROC(fpr_valid,tpr_valid,"ROC curve for valid data")

<div style="background:#FFFFAA">
    <h3>Cross-validation performance</h3>
    <p>
The participants do not have access to the labels Y_valid and Y_test to self-assess their validation and test performances. But training performance is not a good prediction of validation or test performance. Using cross-validation, the training data is split into multiple training/test folds, which allows participants to self-assess their model during development. The average CV result and 95% confidence interval is displayed.
   </div>

In [47]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
scores = cross_val_score(M, X_train, Y_train, cv=5, scoring=make_scorer(scoring_function))
print('\nCV score (95 perc. CI): %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

FIT: dim(X)= [52684, 200]
FIT: dim(y)= [52684, 1]
PREDICT: dim(X)= [13172, 200]
PREDICT: dim(y)= [13172, 1]
FIT: dim(X)= [52685, 200]
FIT: dim(y)= [52685, 1]
PREDICT: dim(X)= [13171, 200]
PREDICT: dim(y)= [13171, 1]
FIT: dim(X)= [52685, 200]
FIT: dim(y)= [52685, 1]
PREDICT: dim(X)= [13171, 200]
PREDICT: dim(y)= [13171, 1]
FIT: dim(X)= [52685, 200]
FIT: dim(y)= [52685, 1]
PREDICT: dim(X)= [13171, 200]
PREDICT: dim(y)= [13171, 1]
FIT: dim(X)= [52685, 200]
FIT: dim(y)= [52685, 1]
PREDICT: dim(X)= [13171, 200]
PREDICT: dim(y)= [13171, 1]

CV score (95 perc. CI): 0.91 (+/- 0.00)





<div style="background:#FFFFAA">
<h1> Step 3: Making a submission </h1> 

<h2> Unit testing </h2> 

It is <b><span style="color:red">important that you test your submission files before submitting them</span></b>. All you have to do to make a submission is modify the file <code>model.py</code> in the <code>sample_code_submission/</code> directory, then run this test to make sure everything works fine. This is the actual program that will be run on the server to test your submission. 
<br>
Keep the sample code simple.
</div>

In [48]:
!python $problem_dir/ingestion.py $data_dir $result_dir $problem_dir $model_dir

Using input_dir: C:\Users\isabe\Pictures\monet-master\monet-master\public_data
Using output_dir: C:\Users\isabe\Pictures\monet-master\monet-master\starting_kit\sample_result_submission\
Using program_dir: C:\Users\isabe\Pictures\monet-master\monet-master\starting_kit\ingestion_program\
Using submission_dir: C:\Users\isabe\Pictures\monet-master\monet-master\starting_kit\sample_code_submission\

========== Ingestion program version 6 ==========

************************************************
******** Processing dataset Perso ********
************************************************
========= Reading and converting data ==========
Info file found : C:\Users\isabe\Pictures\monet-master\monet-master\public_data\perso_public.info
========= Reading C:\Users\isabe\Pictures\monet-master\monet-master\public_data\perso_feat.type
[+] Success in  0.00 sec
========= Reading C:\Users\isabe\Pictures\monet-master\monet-master\public_data\perso_train.data
Replace missing values by 0 (slow, sorry)
[+] 

<div style="background:#FFFFAA">
Also test the scoring program:
    </div>

In [49]:
scoring_output_dir = '../scoring_output_dir'
!python $score_dir/score.py $data_dir $result_dir $scoring_output_dir

======= Set 1 (Perso_train): roc_auc_score(set1_score)=0.995460074196 =======



<div style="background:#FFFFAA">
    <h1> Preparing the submission </h1>

Zip the contents of `sample_code_submission/` (without the directory), or download the challenge public_data and run the command in the previous cell, after replacing sample_data by public_data.
Then zip the contents of `sample_result_submission/` (without the directory).
<b><span style="color:red">Do NOT zip the data with your submissions</span></b>.

In [50]:
import datetime 
from data_io import zipdir
the_date = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
sample_code_submission = 'sample_code_submission_' + the_date + '.zip'
sample_result_submission = 'sample_result_submission_' + the_date + '.zip'
zipdir(sample_code_submission, model_dir)
zipdir(sample_result_submission, result_dir)
print("Submit one of these files:\n" + sample_code_submission + "\n" + sample_result_submission)

Submit one of these files:
sample_code_submission_19-04-08-00-01.zip
sample_result_submission_19-04-08-00-01.zip
